In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb

from collections import OrderedDict
from sklearn.metrics import roc_auc_score
from copy import deepcopy

from autowoe import ReportDeco, AutoWoE

### Чтение и подготовка обучающей выборки

In [ ]:
train = pd.read_csv("./example_data/train_demo.csv",
                    low_memory=False,
                    index_col="line_id",
                    parse_dates = ["datetime_" + str(i) for i in range(2)],)

### Чтение и подготовка тестовой выборки

In [ ]:
test = pd.read_csv("./example_data/test_demo.csv",
                   index_col="line_id", 
                   parse_dates = ["datetime_" + str(i) for i in range(2)])
    
test_target = pd.read_csv("./example_data/test-target_demo.csv")["target"]
test["target"] = test_target.values

### Параметры модели

Для обучения модели рекомендуется указать тип признаков для обучения.
Поэтому создается словарь features_type с ключами: 

 "real" -- вещественный признак
 
 "cat" --  категориальный.
 
 __"date"-- ("%Y%d%m", ("m", "d", "wd", "h", "min"))__
 
 Для признаков, которые не размечены, типы будут определены автоматом. Такой вариант будет работать, но качество порядочно просядет
 
__Попробуем указать даты с форматом None (автоопределение) и сезонностью - день месяца и день недели__

#### features_type

In [ ]:
num_col = list(filter(lambda x: "numb" in x, train.columns))
num_feature_type = {x: "real" for x in num_col}

date_col = list(filter(lambda x: "datetime" in x, train.columns))
date_feature_type = {x: (None, ("d", "wd")) for x in date_col}

In [ ]:
features_type = dict(**num_feature_type,
                     **date_feature_type
                    )

In [ ]:
# подробно параметры описаны в Example_1
auto_woe = AutoWoE(monotonic=True,
                     max_bin_count=4,
                     oof_woe=False,
                     regularized_refit=False,
                     p_val=0.05,
                     debug=False,
                     verbose=0
        )
auto_woe = ReportDeco(auto_woe)

In [ ]:
auto_woe.fit(train[num_col + date_col + ['target']], 
             target_name="target",
             features_type=features_type,
            )

In [ ]:

pred = auto_woe.predict_proba(test)
roc_auc_score(test['target'], pred)

##### Замечание
ReportDeco - обертка для построения отчета. Она не обязательна для обучения и применения модели, но обязательна для построения отчета (см последнюю ячейку).

### Значения коэфициентов и p-values

При указании regularized_refit=False будет произведена оценка p-value на коэфициенты модели. Коэфициенты с p-value выше указанного порога не будут включены в модель

In [ ]:
auto_woe.features_fit

In [ ]:
auto_woe.intercept

In [ ]:
auto_woe.p_vals

### Формирование отчета

In [ ]:
report_params = {"automl_date_column": "report_month", # колонка с датой в формате params['datetimeFormat']
                 "output_path": "./AUTOWOE_REPORT_2", # папка, куда сгенерится отчет и сложатся нужные файлы
                 "report_name": "___НАЗВАНИЕ ОТЧЕТА___",
                 "report_version_id": 1,
                 "city": "Воронеж",
                 "model_aim": "___ЦЕЛЬ ПОСТРОЕНИЯ МОДЕЛИ___",
                 "model_name": "___НАЗВАНИЕ МОДЕЛИ___",
                 "zakazchik": "___ЗАКАЗЧИК___",
                 "high_level_department": "___ПОДРАЗДЕЛЕНИЕ___",
                 "ds_name": "___РАЗРАБОТЧИК МОДЕЛИ___",
                 "target_descr": "___ОПИСАНИЕ ЦЕЛЕВОГО СОБЫТИЯ___",
                 "non_target_descr": "___ОПИСАНИЕ НЕЦЕЛЕВОГО СОБЫТИЯ___"}

auto_woe.generate_report(report_params)